In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/24 20:05:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/24 20:05:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/24 20:05:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
appearances = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/ejercicio_s8/appearances.csv')
clubs = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/ejercicio_s8/clubs.csv')
competitions = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/ejercicio_s8/competitions.csv')
games = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/ejercicio_s8/games.csv')
leagues = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/ejercicio_s8/leagues.csv')
players = spark.read.option("header", "true").option('inferSchema', "true").csv('./data/ejercicio_s8/players.csv')

In [10]:
appearances.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)



In [34]:
from pyspark.sql.functions import count, desc
players.na.drop(subset=['country_of_birth']).groupBy('country_of_birth').agg(count('player_id').alias('no_de_jugadores')).orderBy(desc('no_de_jugadores')).limit(3).show()

+----------------+---------------+
|country_of_birth|no_de_jugadores|
+----------------+---------------+
|          France|           1694|
|           Spain|           1388|
|           Italy|           1312|
+----------------+---------------+



In [12]:
from pyspark.sql.functions import col

df_rojas = appearances.join(players, 'player_id')

df_rojas.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- current_club_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- position: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- market_value_in_gbp: double (nullable = true)
 |-- highest_market_value_in_gbp: double (nullable = true)
 |-- url: string

In [21]:
from pyspark.sql.functions import sum

df_rojas.where(col('red_cards') > 0).groupBy('pretty_name').agg(sum('red_cards').alias('tarjetas_rojas')).orderBy(desc('tarjetas_rojas')).show()

+----------------+--------------+
|     pretty_name|tarjetas_rojas|
+----------------+--------------+
|         Marcelo|             6|
|         Rodrigo|             5|
|Domenico Berardi|             5|
|      Thomas Lam|             5|
|          Hilton|             5|
|    Granit Xhaka|             5|
| Mario Balotelli|             5|
|   Heung Min Son|             4|
| Gabriel Paletta|             4|
|          Rafael|             4|
|      Joao Pedro|             4|
| Damien Da Silva|             4|
|          Felipe|             4|
|           Naldo|             4|
|      Jordi Amat|             4|
|     Ivan Ordets|             4|
|          Danilo|             4|
| Alexander Djiku|             4|
| Rodrigo De Paul|             4|
| Roberto Soriano|             4|
+----------------+--------------+
only showing top 20 rows



In [27]:
df_premier_league = games.join(competitions, games['competition_code'] == competitions['competition_id']).where(col('name') == 'premier-league')

In [24]:
df_premier_league.printSchema()

root
 |-- competition_id: string (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country_id: integer (nullable = true)
 |-- country_name: string (nullable = true)
 |-- domestic_league_code: string (nullable = true)
 |-- confederation: string (nullable = true)
 |-- url: string (nullable = true)



In [29]:
df_premier_league.groupBy(competitions['name']).count().show()

+--------------+-----+
|          name|count|
+--------------+-----+
|premier-league| 2809|
+--------------+-----+



In [30]:
df_asistencia = games.join(competitions, games['competition_code'] == competitions['competition_id']) \
    .join(leagues, competitions['domestic_league_code'] == leagues['league_id'])

In [33]:
df_asistencia.groupBy(leagues['name']).agg(count('game_id').alias('no_juegos')).orderBy(desc('no_juegos')).show(3)

+--------------+---------+
|          name|no_juegos|
+--------------+---------+
|  premier-liga|     4275|
|        laliga|     3706|
|premier-league|     3547|
+--------------+---------+
only showing top 3 rows

